#### Packages

In [264]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt

#### Read Data

In [265]:
store_info = pd.read_excel('OR_hw01_data.xlsx', 'Store')
stores = range(len(store_info['Store ID']))
expected_daily_demand = store_info['Expected Daily Demand (unit)']
store_xcoord = store_info['x-coordinate (km)']
store_ycoord = store_info['y-coordinate (km)']

dc_info = pd.read_excel('OR_hw01_data.xlsx', 'DC')
dcs = range(len(dc_info['Location ID']))
maintenance_costs = dc_info['Maintenance Cost ($/unit)']
construction_costs = dc_info['Construction Cost ($)']
dc_xcoord = dc_info['x-coordinate (km)']
dc_ycoord = dc_info['y-coordinate (km)']
dc_maxscale = dc_info['Maximum Scale (unit)']

S = 1 # replenishment cost

#### Model 1(a): min TC (Multiple sourcing)

In [266]:
eg1a = Model("eg1a")

#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg1a.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))
    
# sj = the scale level of a DC built at loc j
s = []
for j in dcs:
    s.append(eg1a.addVar(lb=0, vtype = GRB.INTEGER, name = "s" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# zij = 1 if products are replenished by DCj to store i (rij>0)
z = []
for i in stores:
    z.append([])
    for j in dcs:
        z[i].append(eg1a.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

##### Objective Fucntion

In [267]:
eg1a.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(v[j] * s[j] * maintenance_costs[j] for j in dcs) \
    + quicksum(quicksum(S * r[i][j] * (wx[i][j] + wy[i][j]) for j in dcs) for i in stores)\
    ,GRB.MINIMIZE
)

##### Constraints

In [268]:
eg1a.addConstrs((quicksum(v[j] * r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg1a.addConstrs((quicksum(r[i][j] for i in stores) == s[j] for j in dcs), "DC scale lv == its total replenishment q")
eg1a.addConstrs((s[j] <= v[j] * dc_maxscale[j] for j in dcs), "DC scale lv <= its max scale lv")
eg1a.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment >= 0")

#eg1a.addConstrs((quicksum(r[i][j] for i in stores) <= v[j] * 20000 for j in dcs), "replenishment amount exists when vj=1")
eg1a.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg1a.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg1a.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg1a.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

##### Optimize

In [269]:
eg1a.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4030 rows, 4020 columns and 6030 nonzeros
Model fingerprint: 0x870ffba5
Model has 2010 quadratic objective terms
Model has 100 quadratic constraints
Variable types: 0 continuous, 4020 integer (1010 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+03, 8e+03]
  QObjective range [2e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
  QRHS range       [1e+02, 2e+02]
Presolve removed 4010 rows and 3000 columns
Presolve time: 0.01s
Presolved: 3130 rows, 2030 columns, 9060 nonzeros
Variable types: 0 continuous, 2030 integer (10 binary)
Found heuristic solution: objective 623271.00000

Root relaxation: objective 4.864806e+05, 165 iterations, 0.01 

In [270]:
print("1(a) Results | Multiple sourcing: ")

for j in dcs:
    print(v[j].varName, '=', int(v[j].x))

for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(int(r[i][j].x), "\t", end="")
        else:
            print(int(r[i][j].x), "", end="")
    print("")

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg1a.ObjVal)

1(a) Results | Multiple sourcing: 
v1 = 1
v2 = 1
v3 = 1
v4 = 1
v5 = 1
v6 = 1
v7 = 1
v8 = 0
v9 = 1
v10 = 1
DC1 	0 	154 	0 	0 	0 	0 	0 	0 	0 	144 	0 	0 	0 	0 	0 	130 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	0 	0 	114 	0 	107 	0 	184 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	171 	0 	0 	149 	0 	0 	0 	0 	154 	131 	178 	152 	0 	0 	0 	0 	0 	0 	152 	0 	0 	0 	0 	122 	0 	189 	109 	0 	0 	0 	0 	43 	0 	0 	0 	0 	0 	191 	0 	0 	178 	0 	122 	0 	0 	118 	
DC2 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	200 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	0 	195 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	186 	0 	0 	0 	132 	0 	0 	138 	0 	0 	0 	0 	125 	0 	0 	0 	0 	0 	0 	0 	0 	0 	
DC3 	0 	0 	0 	138 	128 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	171 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	34 	0 	0 	0 	0 	0 	0 	112 	0 	0 	

#### Model 1(b): min TC (Single sourcing)

In [271]:
eg1b = Model("eg1b")
eg1b.Params.LogFile = "eg1b.log" 
results = glt.parse("eg1b*.log")
summary = results.summary()
nodelog_progress = results.progress("nodelog")


#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg1b.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# sj = the scale level of a DC built at loc j
s = []
for j in dcs:
    s.append(eg1b.addVar(lb=0, vtype = GRB.INTEGER, name = "s" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# zij = 1 if products are replenished by DCj to store i (rij>0)
z = []
for i in stores:
    z.append([])
    for j in dcs:
        z[i].append(eg1b.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

Set parameter LogFile to value "eg1b.log"


##### Objective Function

In [272]:
eg1b.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(v[j] * s[j] * maintenance_costs[j] for j in dcs) \
    + quicksum(quicksum(r[i][j] * (wx[i][j] + wy[i][j]) for i in stores) for j in dcs)\
    ,GRB.MINIMIZE
)

##### Constraints

In [273]:
eg1b.addConstrs((quicksum(z[i][j] * r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg1b.addConstrs((quicksum(z[i][j] for j in dcs) == 1 for i in stores), "每間商店都只能被一個DC去補")

eg1b.addConstrs((quicksum(r[i][j] for i in stores) == s[j] for j in dcs), "DC scale lv == its total replenishment q")
eg1b.addConstrs((s[j] <= v[j] * dc_maxscale[j] for j in dcs), "DC scale lv <= its max scale lv")
eg1b.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment amount >= 0")

eg1b.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg1b.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg1b.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg1b.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

##### Optimize

In [274]:
eg1b.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4130 rows, 4020 columns and 7030 nonzeros
Model fingerprint: 0x0fda1383
Model has 2010 quadratic objective terms
Model has 100 quadratic constraints
Variable types: 0 continuous, 4020 integer (1010 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+03, 8e+03]
  QObjective range [2e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
  QRHS range       [1e+02, 2e+02]
Presolve removed 4010 rows and 2000 columns
Presolve time: 0.01s
Presolved: 3230 rows, 3030 columns, 10060 nonzeros
Variable types: 0 continuous, 3030 integer (1010 binary)
Found heuristic solution: objective 945624.00000

Root relaxation: objective 4.863800e+05, 136 iterations, 0.

##### Results

In [275]:
print("1(b) Results | Single sourcing: ")

for j in dcs:
    print(v[j].varName, '=', int(v[j].x))

replenish_plan = []   
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(int(r[i][j].x), "\t", end="")
        else:
            print(int(r[i][j].x), "", end="")
    print("")

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg1b.ObjVal)

1(b) Results | Single sourcing: 
v1 = 1
v2 = 1
v3 = 1
v4 = 1
v5 = 1
v6 = 1
v7 = 1
v8 = 0
v9 = 1
v10 = 1
DC1 	0 	154 	0 	0 	0 	0 	0 	0 	0 	144 	0 	0 	0 	0 	0 	130 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	0 	0 	114 	0 	0 	0 	184 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	120 	0 	0 	0 	0 	0 	0 	0 	0 	154 	131 	178 	152 	0 	0 	0 	0 	0 	150 	152 	0 	0 	0 	0 	122 	0 	189 	109 	0 	0 	0 	0 	181 	0 	0 	0 	0 	0 	191 	0 	0 	178 	0 	122 	0 	0 	0 	
DC2 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	200 	0 	0 	107 	0 	0 	0 	0 	0 	0 	0 	171 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	195 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	186 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	0 	125 	0 	0 	0 	0 	0 	0 	0 	0 	118 	
DC3 	0 	0 	0 	138 	128 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	166 	0 	0 	0 	0 	0 	0 	112 	0 	0 	0 	0

#### Model 2: min the maximum distance

In [276]:
eg2 = Model("eg2")
B = int #cost_constraint

#-------- Add variables as a list ---------#

# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg2.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# sj = the scale level of a DC built at loc j
s = []
for j in dcs:
    s.append(eg2.addVar(lb=0, vtype = GRB.INTEGER, name = "s" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))


# wij=1 if the DCj is the closest to store i
w = []
for i in stores:
    w.append([])
    for j in dcs:
        w[i].append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = "w" + str(i+1) + str(j+1)))

d = eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "d")
B = eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "B")

##### Objective Function

In [277]:
eg2.setObjective(d,GRB.MINIMIZE)

##### Constraints

In [278]:
eg2.addConstrs((quicksum(v[j] * r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg2.addConstrs((quicksum(r[i][j] for i in stores) == s[j] for j in dcs), "DC scale lv == its total replenishment q")
eg2.addConstrs((s[j] <= v[j] * dc_maxscale[j] for j in dcs), "DC scale lv <= its max scale lv")
eg2.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment >= 0")

eg2.addConstrs((construction_costs[j] + maintenance_costs[j] * s[j] <= B for j in dcs), "TC cannot > B")
eg2.addConstrs((quicksum(w[i][j] for j in dcs) == 1 for i in stores), "每間store都有一個最近的DC")
eg2.addConstrs((w[i][j] <= v[j] for i in stores for j in dcs), "要確定那間DC有蓋")
eg2.addConstrs((d >= quicksum(w[i][j] * (wx[i][j] + wy[i][j]) for j in dcs) for i in stores), "每間store到最近的DC的最遠距離")
eg2.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg2.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg2.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg2.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

##### Optimize

In [279]:
eg2.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5140 rows, 4022 columns and 9050 nonzeros
Model fingerprint: 0xeea8e159
Model has 200 quadratic constraints
Variable types: 0 continuous, 4022 integer (1010 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+03]
  QRHS range       [1e+02, 2e+02]
Presolve removed 4030 rows and 11 columns
Presolve time: 0.04s
Presolved: 8310 rows, 11011 columns, 24110 nonzeros
Presolved model has 4000 SOS constraint(s)
Variable types: 0 continuous, 11011 integer (3010 binary)
Found heuristic solution: objective 51.0000000

Explored 1 nodes (0 simplex iterations) in 0.20 seconds (0.

##### Results

In [280]:
print("3 Results | Multiple sourcing: ")

for j in dcs:
    print(v[j].varName, '=', int(v[j].x))

replenish_plan = []   
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(int(r[i][j].x), "\t", end="")
        else:
            print(int(r[i][j].x), "", end="")
    print("")

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg2.ObjVal)

3 Results | Multiple sourcing: 
v1 = 1
v2 = 1
v3 = 1
v4 = 1
v5 = 1
v6 = 1
v7 = 1
v8 = 1
v9 = 1
v10 = 1
DC1 	0 	154 	0 	0 	128 	0 	161 	0 	0 	144 	0 	135 	0 	0 	142 	130 	0 	0 	0 	0 	0 	0 	0 	175 	0 	0 	0 	0 	0 	0 	0 	164 	0 	0 	0 	0 	0 	0 	150 	174 	0 	135 	0 	166 	0 	135 	0 	0 	0 	0 	0 	0 	143 	0 	171 	0 	0 	0 	0 	166 	0 	0 	154 	0 	0 	0 	0 	0 	0 	157 	0 	0 	0 	0 	0 	0 	0 	122 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	118 	
DC2 	0 	0 	0 	0 	0 	0 	17 	64 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	132 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	184 	0 	0 	0 	0 	0 	0 	178 	0 	0 	0 	187 	0 	0 	0 	0 	0 	169 	0 	0 	0 	0 	0 	0 	131 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	178 	0 	0 	0 	0 	0 	
DC3 	0 	0 	0 	0 	0 	0 	0 	127 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	183 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	0 	159 	0 	0 	0 	0 	0 	0 	0 	0 	195 	0 	0 	0 	0

#### Visualization

In [281]:
glt.plot(results.summary())


interactive(children=(Dropdown(description='x', options=('Platform', 'Time', 'PhysicalCores', 'LogicalProcesso…